In [1]:
import numpy as np
# import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
import cv2
import os
import dlib

In [2]:
import tensorflow as tf

# Check if CUDA is available
if tf.test.is_built_with_cuda():
    print("CUDA is available")
    # Enable GPU memory growth (optional)
    physical_devices = tf.config.list_physical_devices('GPU')
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, False)
else:
    print("CUDA is not available")

# Check TensorFlow version
print("TensorFlow version:", tf.__version__)

CUDA is available
TensorFlow version: 2.10.0


In [ ]:
# import zipfile
# zip_ref = zipfile.ZipFile('fercnn.zip', 'r')
# zip_ref.extractall('/content')
# zip_ref.close()

In [7]:
import cv2
import os
import numpy as np

train_path = r'E:\project\KDEF\train'
val_path = r'E:\project\KDEF\test'


# Define desired image dimensions
img_height = 224
img_width = 224

def load_images_from_folder(folder):
    images = []
    labels = []
    for emotion_folder in os.listdir(folder):
        emotion_path = os.path.join(folder, emotion_folder)
        if os.path.isdir(emotion_path):
            for image_filename in os.listdir(emotion_path):
                img_path = os.path.join(emotion_path, image_filename)
                img = cv2.imread(img_path)  # Use cv2 to read images
                if img is not None:
                    # Resize image to desired dimensions if needed
                    img = cv2.resize(img, (img_height, img_width))
                    images.append(img)
                    labels.append(emotion_folder)  # Assuming folder names are the labels
    return np.array(images), np.array(labels)

X_train, y_train = load_images_from_folder(train_path)
X_val, y_val = load_images_from_folder(val_path)


In [8]:
def save_images_to_folder(images, labels, output_folder):
    # Iterate through each image and label pair
    for image, label in zip(images, labels):
        # Create the corresponding label folder in the output folder
        output_label_folder = os.path.join(output_folder, label)
        os.makedirs(output_label_folder, exist_ok=True)

        # Save the image to the output folder
        output_path = os.path.join(output_label_folder, f"{len(os.listdir(output_label_folder))}.jpg")
        cv2.imwrite(output_path, image)

# Define the output folders
output_train_folder = r'E:\KDEF\ResizedKDEF_train'
output_val_folder = r'E:\KDEF\ResizedKDEF_validation'

# Save the training and validation images to the output folders
save_images_to_folder(X_train, y_train, output_train_folder)
save_images_to_folder(X_val, y_val, output_val_folder)


In [9]:


# Load the pre-trained shape predictor model
predictor_path = 'shape_predictor_68_face_landmarks.dat'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

# Function to detect facial landmarks in an image
def detect_landmarks(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Detect faces in the grayscale image
    faces = detector(gray)
    landmarks = []
    # Iterate through each detected face
    for face in faces:
        # Detect facial landmarks
        shape = predictor(gray, face)
        # Convert the landmarks to a list of tuples
        landmarks.append([(shape.part(i).x, shape.part(i).y) for i in range(shape.num_parts)])
    return landmarks

def apply_landmark_detection(input_folder, output_folder):
    # Iterate through each subfolder (emotion) in the input folder
    for emotion_folder in os.listdir(input_folder):
        emotion_folder_path = os.path.join(input_folder, emotion_folder)
        if os.path.isdir(emotion_folder_path):
            # Create the corresponding emotion folder in the output folder
            output_emotion_folder = os.path.join(output_folder, emotion_folder)
            os.makedirs(output_emotion_folder, exist_ok=True)

            # Iterate through each image in the emotion folder
            for filename in os.listdir(emotion_folder_path):
                if filename.endswith('.jpg') or filename.endswith('.png'):
                    # Read the image
                    image_path = os.path.join(emotion_folder_path, filename)
                    image = cv2.imread(image_path)

                    # Detect landmarks in the image
                    landmarks = detect_landmarks(image)

                    # Process or analyze the landmarks as needed
                    # For example, you can draw the landmarks on the image and save it
                    for landmark_set in landmarks:
                        for landmark in landmark_set:
                            cv2.circle(image, landmark, 1, (0, 0, 255), -1)  # Draw a red dot at each landmark

                    # Save the image with landmarks in the output folder
                    output_path = os.path.join(output_emotion_folder, f"landmarked_{filename}")
                    cv2.imwrite(output_path, image)

# Set the input and output folders for resized train and test images
input_resized_train_folder = r'E:\KDEF\ResizedKDEF_train'
input_resized_test_folder = r'E:\KDEF\ResizedKDEF_validation'
output_landmarked_train_folder = r'E:\KDEF\landmarked_ResizedKDEF_train'
output_landmarked_test_folder = r'E:\KDEF\landmarked_ResizedKDEF_validation'

# Apply landmark detection to resized train and test images and save in new folders
apply_landmark_detection(input_resized_train_folder, output_landmarked_train_folder)
apply_landmark_detection(input_resized_test_folder, output_landmarked_test_folder)


In [3]:
# input_resized_train_folder = '/content/resized_train'
# input_resized_test_folder = '/content/resized_val'
output_landmarked_train_folder = '/inception/resized_train'
output_landmarked_test_folder = '/inception/resized2_val'